In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Load and preprocess data
df = pd.read_csv("/kaggle/input/tmnist-alphabet-94-characters/94_character_TMNIST.csv")
X = df.drop(['names', 'labels'], axis=1).values.reshape(df.shape[0], 28, 28, 1) / 255
y = df['labels']

chars = sorted(list(df['labels'].unique()))
vocab_size = len(chars)

itos = {i: ch for i, ch in enumerate(chars)}
stoi = {ch: i for i, ch in enumerate(chars)}

y_upd = np.array([stoi[ch] for ch in y])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_upd, test_size=0.2, random_state=1337)

# Convert labels to one-hot encoding
y_train_one_hot = to_categorical(y_train, num_classes=vocab_size)
y_test_one_hot = to_categorical(y_test, num_classes=vocab_size)

# Define the model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128),
    BatchNormalization(),
    ReLU(),
    Dense(vocab_size, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=3e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train_one_hot, batch_size=128, epochs=20, validation_data=(X_test, y_test_one_hot))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test_one_hot)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")
